In [1]:
import os
from gensim.models import KeyedVectors
import json
from simhash import Simhash, SimhashIndex
import numpy as np

C:\Users\Mansi\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#webhoseio.config(token=os.environ['WEBHOSE_TOKEN'])
#query_params = {
#    "q": "organization:Tesla",
#    "ts": "1523748602856",
#    "sort": "crawled"
#}

## Load Word Vector Model¶

In [2]:
model_path = 'C:\\Users\\Mansi\\Documents\\Harrisburg classes\\Sem 5\\610\\HW7\\GoogleNews-vectors-negative300.bin\\'

##'/Users/javidbeck/projects/data/model/'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_w2v_AP    = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin', True)
#model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [3]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [4]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [5]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

## Load the JSon file and print original dataset titles

In [6]:
## reading the json file
with open('C:\\Users\\Mansi\\Documents\\Harrisburg classes\\Sem 7 - 699\\ProjectCollaterals_610_MansiJoshi_20180817\\data_feedsNGG.json') as json_datafeeds:
    load_datafeeds = json.load(json_datafeeds)

In [7]:
feeds = []
i = 0
for feed in load_datafeeds:
    feed['id'] = str(i)
    print(feed['id'], str(feed['title']))
    i += 1
    feeds.append(feed)

0 Top 10 Canadian Stocks To Invest In 2019
1 National Grid : NG submits response to RIIO T2 consultation
2 National Grid : Ofgem Proposals 'a Step in the Right Direction' | MarketScreener
3 National Grid : Ofgem Proposals 'a Step in the Right Direction'
4 National Grid : NG submits response to RIIO T2 consultation | MarketScreener
5 National Grid Submits Response To Ofgem On RIIO T2 Consultation
6 National Grid Submits Response To Ofgem On RIIO T2 Consultation
7 National Grid criticises new UK power regulation plan
8 Most Traded Stocks by Value at 10:00
9 Most Traded Stocks by Value at 11:00
10 Most Traded Stocks by Value at 12:00
11 Most Traded Stocks by Value at 13:00
12 Most Traded Stocks by Value at 15:00
13 Most Traded Stocks by Value at 14:00
14 Most Traded Stocks by Value at 16:00
15 National Grid plc (NGG) Stake Lowered by Mirae Asset Global Investments Co. Ltd.
16 Most Traded Stocks by Value at 17:00
17 Private Advisor Group LLC Decreases Stake in National Grid plc (NGG)
18 Na

373 BEDC connects Omozogie- Uteh community to national grid
374 PG&E Announces New Chief Executive Officer and Appointment of a Refreshed Board of Directors; New Leadership Focused on Enhancing Safety Culture and Operational Excellence
375 PG&E Announces New Chief Executive Officer and Appointment of a Refreshed Board of Directors; New Leadership Focused on Enhancing Safety Culture and Operational Excellence
376 LAYLA：반도체관련 장비 및 부품의 종합시장(58)
377 Company Market Strategy
378 National Grid Plc -- announces completion of a periodic review of ratings of National Grid Plc
379 British Transco International Finance B.V. -- announces completion of a periodic review of ratings of National Grid Gas Plc
380 National Grid Electricity Transmission plc -- Moody's announces completion of a periodic review of ratings of National Grid Electricity Transmission plc
381 PG&E Announces New Chief Executive Officer and Appointment of a Refreshed Board of Directors; New Leadership Focused on Enhancing Safety C

## Define distance, index using SimHash, calculate duplicates subset using SimHash only

In [8]:
distance = 5
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=distance)

In [9]:
# Select a feed from the list based on its index
SELECTED_INDEX = 32 #27
feed_sel = feeds[SELECTED_INDEX]
print(feed_sel['title'])
# calculate  hash value
feed_hash = Simhash(str(feed_sel['title']))
# find all duplicate indices
dup_indices = index.get_near_dups(feed_hash)
print("Number of duplicates (SimHash): " + str(len(dup_indices)))

Most Traded Stocks by Value at 10:00
Number of duplicates (SimHash): 12


## Print SimHash-based duplicate titles

In [10]:
for dupi in dup_indices:
    print(feeds[int(dupi)]['title'])

Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00
Most Traded Stocks by Value at 10:00


In [11]:
## Identifying duplicates across the full datasets and deduping the datasets to include only unique values

full=[feed['id'] for feed in feeds]
deduped=[]


while len(full) > 0:
    SELECTED_INDEX =  int(full[0]) 
    feed_sel = feeds[SELECTED_INDEX]
    print("Number of titles: " + str(len(full))+ " | " + feed_sel['title'])
    
    feed_hash = Simhash(str(feed_sel['title']))      # Simhash value
    dup_indices = index.get_near_dups(feed_hash)      # duplicates
    
    print("Number of duplicates (SimHash): " + str(len(dup_indices)))

    print("Number of deduplicated results: " + str(len(deduped)))

    full = list(set(full) - set(dup_indices))
    deduped.append(SELECTED_INDEX)



Number of titles: 460 | Top 10 Canadian Stocks To Invest In 2019
Number of duplicates (SimHash): 1
Number of deduplicated results: 0
Number of titles: 459 | Eni plans third development well for Blacktip gas field
Number of duplicates (SimHash): 1
Number of deduplicated results: 1
Number of titles: 458 | Oil, Gas, And Fracking News Reads: 31March, 2019 - Part 1
Number of duplicates (SimHash): 1
Number of deduplicated results: 2
Number of titles: 457 | Jane Street Group LLC Purchases 1,044,909 Shares of National Grid plc (NGG)
Number of duplicates (SimHash): 1
Number of deduplicated results: 3
Number of titles: 456 | National Grid (NGG) vs. Archrock (AROC) Head to Head Survey
Number of duplicates (SimHash): 2
Number of deduplicated results: 4
Number of titles: 454 | PG&E Announces New Chief Executive Officer and Appointment of a Refreshed Board of Directors; New Leadership Focused on Enhancing Safety Culture and Operational Excellence
Number of duplicates (SimHash): 10
Number of deduplic

Number of titles: 166 | National Grid : Board Change
Number of duplicates (SimHash): 1
Number of deduplicated results: 133
Number of titles: 165 | Anglo-Eastern Plantations (AEP) Sets New 52-Week Low at $512.00
Number of duplicates (SimHash): 1
Number of deduplicated results: 134
Number of titles: 164 | Logan Capital Management Inc. Sells 663 Shares of National Grid plc (NGG)
Number of duplicates (SimHash): 1
Number of deduplicated results: 135
Number of titles: 163 | Head to Head Comparison: National Grid (NGG) and Tallgrass Energy (TGE)
Number of duplicates (SimHash): 1
Number of deduplicated results: 136
Number of titles: 162 | National Grid Plc -- announces completion of a periodic review of ratings of National Grid Plc
Number of duplicates (SimHash): 1
Number of deduplicated results: 137
Number of titles: 161 | Broker Forecast - UBS issues a broker note on National Grid PLC
Number of duplicates (SimHash): 1
Number of deduplicated results: 138
Number of titles: 160 | Reference | Ex

Number of duplicates (SimHash): 1
Number of deduplicated results: 214
Number of titles: 76 | Elumelu 'To Add' 1,400mw To National Grid With $2.5bn Investment In Power » Naijatweet
Number of duplicates (SimHash): 1
Number of deduplicated results: 215
Number of titles: 75 | Pharmaceuticals, miners lead London markets higher - MarketWatch
Number of duplicates (SimHash): 2
Number of deduplicated results: 216
Number of titles: 73 | National Grid (NGG) versus Archrock Partners (APLP) Head-To-Head Review - Dispatch Tribunal
Number of duplicates (SimHash): 1
Number of deduplicated results: 217
Number of titles: 72 | ElectraLink and Renewable Energy Association launch partnership to champion a decentralised electricity system
Number of duplicates (SimHash): 1
Number of deduplicated results: 218
Number of titles: 71 | iShares Global Utilities ETF (JXI) Rises 0.63% for Apr 6
Number of duplicates (SimHash): 1
Number of deduplicated results: 219
Number of titles: 70 | National Grid (NG) Given New G

In [12]:
print("Total number of deduped results: " + str(len(deduped)))

print("Deduped results:")

deduped.sort()
for i in deduped:
    print(str(i) +' | ' +feeds[int(i)]['title'])

Total number of deduped results: 287
Deduped results:
0 | Top 10 Canadian Stocks To Invest In 2019
1 | National Grid : NG submits response to RIIO T2 consultation
2 | National Grid : Ofgem Proposals 'a Step in the Right Direction' | MarketScreener
3 | National Grid : Ofgem Proposals 'a Step in the Right Direction'
4 | National Grid : NG submits response to RIIO T2 consultation | MarketScreener
6 | National Grid Submits Response To Ofgem On RIIO T2 Consultation
7 | National Grid criticises new UK power regulation plan
14 | Most Traded Stocks by Value at 16:00
15 | National Grid plc (NGG) Stake Lowered by Mirae Asset Global Investments Co. Ltd.
17 | Private Advisor Group LLC Decreases Stake in National Grid plc (NGG)
18 | National Grid plc (NGG) Position Lowered by Mirae Asset Global Investments Co. Ltd.
19 | 5,636 Shares in National Grid plc (NGG) Purchased by VisionPoint Advisory Group LLC
20 | VisionPoint Advisory Group LLC Purchases Shares of 5,636 National Grid plc (NGG)
21 | Nation

## Calculate and print SimHash + Word2Vec similarity based duplicate titles

In [15]:
## Identifying duplicates using SimHash and Word2Vec for one selected feed. 

count_dup = 0
for dupi in dup_indices:
    try:
        score = calc_similarity(feed_sel['title'], feeds[int(dupi)]['title'], model_w2v_AP)
    except:
        score = 0
    if score > 0.9:
        count_dup += 1
        title_deduped = feeds[int(dupi)]['title'], count_dup
print("Number of Dupies (SimHash + WordVectors): " + str(count_dup))
print(title_deduped)

Number of Dupies (SimHash + WordVectors): 1
("Apple Safari 11.1.1 - Apple's Web browser. (Free)", 1)


In [23]:
## Identifying duplicates across the full datasets using Word2Vec and Simhash and deduping the datasets to include only unique values

full=[feed['id'] for feed in feeds]
deduped=[]


while len(full) > 0:
    SELECTED_INDEX =  int(full[0]) 
    feed_sel = feeds[SELECTED_INDEX]
    print("Number of titles: " + str(len(full))+ " | " + feed_sel['title'])
    
    feed_hash = Simhash(str(feed_sel['title']))      # Simhash value
    dup_indices = index.get_near_dups(feed_hash)      # duplicates
    
    print("Number of duplicates (SimHash): " + str(len(dup_indices)))

    print("Number of deduplicated results: " + str(len(deduped)))

    full = list(set(full) - set(dup_indices))
    deduped.append(SELECTED_INDEX)
    
    furtherdeduped = []
    count_dup = 0
    
##        for dupi in dup_indices:
##        str1=feed_sel['title']
##        if(str1 ==""):
##            #print("str2 is empty")
##            str1 = "x"

##        str2=feeds[int(dupi)]['title']
##        if(str2 ==""):
##            #print("str2 is empty")
##            str2 = "x"

##        score = calc_similarity(str1,str2, model_w2v_AP)
##        print('    '+ str2 + ' | '+str(score))
        
##        if score > 0.4:
##            dup_indices2.append(dupi)

    try:
        score = calc_similarity(feed_sel['title'], feeds[int(dupi)]['title'], model_w2v_AP)
    except:
        score = 0
    if score > 0.7:
        count_dup += 1
        print(feeds[int(dupi)]['title'])

    print("Number of duplicates (SimHash+word2vec): " + str(len(furtherdeduped)))

    full = list(set(full) - set(furtherdeduped))
    deduped.append(SELECTED_INDEX)

    print("Number of titles after deduping (SimHash+word2vec): " + str(len(furtherdeduped)))
    print("\n***********\n")

Number of titles: 15100 | This week’s top stories: iOS 11.4, new HomePod features, last minute WWDC rumors, more
Number of duplicates (SimHash): 3
Number of deduplicated results: 0
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 15097 | Apple's iOS 12 brings group FaceTime and 'Memoji' to iPhone/iPad; with slew of Siri enhancements
Number of duplicates (SimHash): 1
Number of deduplicated results: 2
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 15096 | Facebook fends off New York Times claims over data access
Number of duplicates (SimHash): 2
Number of deduplicated results: 4
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 15094 | WWDC 2018: Does Apple still care about macOS?
Number of duplicates (SimHash): 2
Number of deduplicated results: 6
Num

Number of duplicates (SimHash): 2
Number of deduplicated results: 80
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14864 | Report: Facebook provided dozens of companies with access to user data
Number of duplicates (SimHash): 1
Number of deduplicated results: 82
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14863 | How To Remove Or Change The ‘sent From My Iphone’ Signature On Mail App Emails
Number of duplicates (SimHash): 3
Number of deduplicated results: 84
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14860 | snarkolepsy
Number of duplicates (SimHash): 1
Number of deduplicated results: 86
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14859 | More iOS apps 

Number of duplicates (SimHash): 1
Number of deduplicated results: 158
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14487 | Apple got even tougher on ad trackers at WWDC
Number of duplicates (SimHash): 15
Number of deduplicated results: 160
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14472 | Apple's new 'Measure' tool for iOS 12 threatens dozens of existing apps already in the App Store
Number of duplicates (SimHash): 6
Number of deduplicated results: 162
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14466 | The secret lives of Apple developersLatest Commentary Today, Breaking Business News Worldwide | Latest Commentary Today, Breaking Business News Worldwide
Number of duplicates (SimHash): 1
Number of deduplicated results: 164
Number of d

Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14260 | Facebook Privacy Issues Arise Again Following New Data-Sharing Report
Number of duplicates (SimHash): 2
Number of deduplicated results: 232
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14258 | Why Apple is having so many problems right now
Number of duplicates (SimHash): 8
Number of deduplicated results: 234
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14250 | Apple introduces macOS Mojave with slew of new features, including Dark Mode
Number of duplicates (SimHash): 3
Number of deduplicated results: 236
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 14247 | WhatsApp is the first major third-party app to add support for Apple CarPlay
Number of dupl

Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 13953 | The Post Most: Five dead, nearly 200 sick in E. coli outbreak from lettuce. And investigators are stumped.
Number of duplicates (SimHash): 1
Number of deduplicated results: 310
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 13952 | iPhone News: Apple CEO Tim Cook Not Worried About Price Increase Due To Trump Tariffs
Number of duplicates (SimHash): 1
Number of deduplicated results: 312
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 13951 | These are the best-designed games on the App Store, according to Apple
Number of duplicates (SimHash): 2
Number of deduplicated results: 314
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Numb

...

Number of duplicates (SimHash): 1
Number of deduplicated results: 16290
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 251 | Apple (NASDAQ:AAPL) is Summit Securities Group LLC’s 2nd Largest Position
Number of duplicates (SimHash): 1
Number of deduplicated results: 16292
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 250 | 2018 iPhone Models to Ship Starting September, Price Starts at $600: Ming Chi Kuo
Number of duplicates (SimHash): 1
Number of deduplicated results: 16294
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 249 | Facebook Collects Data On People Who Don't Even Have An Account
Number of duplicates (SimHash): 2
Number of deduplicated results: 16296
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+wor

Number of duplicates (SimHash): 1
Number of deduplicated results: 16384
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 202 | The best iPhone deals of June 2018
Number of duplicates (SimHash): 1
Number of deduplicated results: 16386
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 201 | Download tvOS 12 Beta for Apple TV 4 & Apple TV 4K – How to
Number of duplicates (SimHash): 1
Number of deduplicated results: 16388
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 200 | Apple Announces macOS 10.14, Competing With Fedora 28 & Ubuntu 18.10 This Fall
Number of duplicates (SimHash): 1
Number of deduplicated results: 16390
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 199 

Number of duplicates (SimHash): 1
Number of deduplicated results: 16494
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 146 | Police seize motorbike on Torpoint ferry
Number of duplicates (SimHash): 1
Number of deduplicated results: 16496
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 145 | Google Assistant Most Used Voice Assistant for Shopping, Outpacing Siri and Alexa | Voicebot
Number of duplicates (SimHash): 1
Number of deduplicated results: 16498
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 144 | MSI's Prestige laptop is more MacBook and less gaming beast
Number of duplicates (SimHash): 1
Number of deduplicated results: 16500
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Nu

Number of duplicates (SimHash): 1
Number of deduplicated results: 16588
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 99 | The Twilight of The Macintosh
Number of duplicates (SimHash): 1
Number of deduplicated results: 16590
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 98 | Apple Joins the Impending Avatar Wars With Memoji -- Road to VR
Number of duplicates (SimHash): 1
Number of deduplicated results: 16592
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 97 | 5 new physical fitness features that will achieve your Apple Watch
Number of duplicates (SimHash): 1
Number of deduplicated results: 16594
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 96 | Fca Holding in 

Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 49 | VPN : How to Watch Norwegian TV Abroad
Number of duplicates (SimHash): 1
Number of deduplicated results: 16690
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 48 | CHATTER AWAY: Overnight Open Thread
Number of duplicates (SimHash): 1
Number of deduplicated results: 16692
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 47 | Apple announces controls to help fight iPhone addiction, plus augmented reality and fitness features - Los Angeles Times
Number of duplicates (SimHash): 1
Number of deduplicated results: 16694
Number of duplicates (SimHash+word2vec): 0
Number of titles after deduping (SimHash+word2vec): 0

***********

Number of titles: 46 | Techeia News Update: June 03, 2018 at 04:29AM
Number

In [21]:
print("Number of Dupies (SimHash + WordVectors): " + str(count_dup))

Number of Dupies (SimHash + WordVectors): 1
